In [ ]:
!pip install opencv-python-headless
!pip install fer
!pip install opencv-python
!pip install deepface


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import base64  # Import base64 for decoding image data
from deepface import DeepFace

# JavaScript code to capture the image from the webcam
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = 'Capture Image';
        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to match the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for the user to click the capture button.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks()[0].stop();

        const dataUrl = canvas.toDataURL('image/jpeg', quality);
        div.remove();
        return dataUrl;
    }
    ''')
    display(js)
    data = eval_js('takePhoto(%f)' % quality)
    binary = np.frombuffer(base64.b64decode(data.split(',')[1]), dtype=np.uint8)
    image = cv2.imdecode(binary, cv2.IMREAD_COLOR)
    return image

# Take photo using webcam
image = take_photo()

# Show the captured image
cv2_imshow(image)

# Analyze emotions using DeepFace
try:
    result = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)

    # Access the first element in the list of results if multiple faces are detected
    if isinstance(result, list):
        result = result[0]

    print("Detected Emotion: ", result['dominant_emotion'])
except Exception as e:
    print(f"Error analyzing emotion: {str(e)}")


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import base64  # Import base64 for decoding image data
from deepface import DeepFace
import time

# Global flag to stop the loop
stop_loop = False

# JavaScript code to capture the image from the webcam and stop the loop
def take_photo_or_stop(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhotoOrStop(quality) {
        const div = document.createElement('div');

        // Capture Image button
        const capture = document.createElement('button');
        capture.textContent = 'Capture Image';
        div.appendChild(capture);

        // Stop button
        const stopButton = document.createElement('button');
        stopButton.textContent = 'Stop';
        div.appendChild(stopButton);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to match the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Capture Image button functionality
        await new Promise((resolve, reject) => {
            capture.onclick = resolve;
            stopButton.onclick = reject;  // Reject if Stop button is clicked
        });

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks()[0].stop();

        const dataUrl = canvas.toDataURL('image/jpeg', quality);
        div.remove();
        return [true, dataUrl];  // Image captured
    }
    ''')

    try:
        display(js)
        data = eval_js('takePhotoOrStop(%f)' % quality)
        return data
    except Exception:
        return [False, None]  # Loop stop requested


# Simulated real-time emotion detection with Stop button
while True:
    # Capture image or stop the loop
    capture_status, dataUrl = take_photo_or_stop()

    if not capture_status:
        print("Stopping the real-time emotion detection loop.")
        break

    # Decode and process the captured image
    binary = np.frombuffer(base64.b64decode(dataUrl.split(',')[1]), dtype=np.uint8)
    image = cv2.imdecode(binary, cv2.IMREAD_COLOR)

    # Show the captured image
    cv2_imshow(image)

    # Analyze emotions using DeepFace
    try:
        result = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)

        # Access the first element in the list of results if multiple faces are detected
        if isinstance(result, list):
            result = result[0]

        print("Detected Emotion: ", result['dominant_emotion'])
    except Exception as e:
        print(f"Error analyzing emotion: {str(e)}")

    # Simulate real-time delay (adjust time.sleep as needed)
    time.sleep(1)
